In [3]:
import sqlite3 as sl

con = sl.connect('my-test.db')



In [4]:
with con:
  con.execute ('''
CREATE TABLE USER (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, name TEXT, age INTEGER
);
''')

In [8]:
sql = 'INSERT INTO USER (id, name, age) values (?, ?, ?)'
data = [
  (1, 'Alice', 21),
  (2, 'Bob', 22),
  (3, 'Chris', 23)
]

In [10]:
with con:
  con.executemany(sql, data)

In [ ]:
with con:
  data = con.execute("SELECT * FROM USER WHERE age <= 22")
for row in data:
  print (row)

In [ ]:
import pandas as pd
df_skill = pd.DataFrame({
  'user_id': [1,1,2,2,3,3,3],
  'skill': ['Network Security', 'Algorithm Development', 'Network Security', 'Java', 'Python', 'Data Science', 'Machine Learning']
})
df_skill

In [ ]:
df_skill.to_sql('SKILL', con)
df = pd.read_sql('''
  SELECT s.user_id, u.name, u.age, s.skill
  FROM USER u LEFT JOIN SKILL s ON u.id = s.user_id
''', con)

df

In [24]:
df.to_sql('USER_SKILL', con)

In [25]:
%load_ext sql
from random import uniform, choice
import time
from datetime import datetime, timedelta
import pandas as pd 
%matplotlib inline

In [ ]:
%sql sqlite:///demo.db3

In [ ]:
%sql SELECT date('now');

In [ ]:
%sql SELECT date('now', 'start of month', '+1 month', '-1 day');

In [ ]:
%%sql sqlite://
DROP TABLE IF EXISTS Temperature;
CREATE TABLE Temperature (Timestamp DATETIME NOT NULL, Temperature NUMERIC NOT NULL)

In [ ]:
%%sql sqlite://

CREATE UNIQUE INDEX id_timestamp ON Temperature (Timestamp);

In [ ]:
def dt(days):
  return timedelta(days=days)
N_rows = 500
now = datetime.now()
for i in range(N_rows):
  timestamp = now - dt(days=(N_rows - i))
  temperature = uniform (18, 26)
  %sql INSERT INTO Temperature VALUES (:timestamp, :temperature);

In [ ]:
%sql SELECT datetime(Timestamp) as Date, Temperature FROM Temperature LIMIT 5

In [ ]:
%%sql sqlite://
SELECT Timestamp as Date, Temperature
FROM Temperature
WHERE Date <= '2022-06-31 14:21:45'

In [ ]:
%%sql sqlite://
SELECT Timestamp as Date, Temperature
FROM Temperature
WHERE CAST (strftime('%Y', Date) as Int) = 2021

In [ ]:
%%sql sqlite://
SELECT strftime('%Y', Timestamp) as Year,
  AVG(Temperature) AS MEAN_TMP,
  MAX(Temperature) AS MAX_TMP,
  MIN(Temperature) AS MIN_TMP
FROM Temperature
GROUP BY strftime('%Y', Timestamp)

In [ ]:
ts = %sql SELECT datetime (Timestamp) as date, Temperature FROM Temperature
df = ts.DataFrame()
df.set_index("date", inplace = True)
df.index = pd.to_datetime(df.index)
df.index

In [ ]:
df.plot(figsize=(12, 6))